In [ ]:
import pandas as pd
import math
from random import * 
# import data sets that will be used
seeds = pd.read_csv("../other_csvs/predictive/NCAATourneySeeds.csv")
reg_season = pd.read_csv("../other_csvs/predictive/RegularSeasonDetailedResults.csv")
regions = pd.read_csv("../other_csvs/predictive/Seasons.csv")
team_names = pd.read_csv("../other_csvs/predictive/Teams.csv")

pd.set_option('display.max_rows',1755)

# adjust data sets to only 2014 and later
seeds = seeds[seeds.Season > 2013]
regions = regions[regions.Season > 2013]
reg_season = reg_season[reg_season.Season >2013]

In [721]:
# Adds the column TeamName to the seeds dataframe
names = []
for team in seeds.TeamID:
    names.append(team_names['TeamName'][team_names['TeamID']==team].values[0])
seeds['TeamName'] = names
seeds.head()


,Season,Seed,TeamID,TeamName
1878,2014,W01,1438,Virginia
1879,2014,W02,1437,Villanova
1880,2014,W03,1235,Iowa St
1881,2014,W04,1277,Michigan St
1882,2014,W05,1153,Cincinnati


In [722]:
# initialize arrays
next_round           = []
roundof32            = []
sweet16              = []
elite8               = []
final4               = []
finals               = []
ncaa_winner          = []
roundof32_actual     = []
sweet16_actual       = []
elite8_actual        = []
final4_actual        = []
finals_actual        = []
ncaa_winner_actual   = []

tourney_order = ['W01','W16','W08','W09','W05','W12','W04','W13','W06','W11','W03','W14','W07','W10','W02','W15',
                 'X01','X16','X08','X09','X05','X12','X04','X13','X06','X11','X03','X14','X07','X10','X02','X15',
                 'Y01','Y16','Y08','Y09','Y05','Y12','Y04','Y13','Y06','Y11','Y03','Y14','Y07','Y10','Y02','Y15',
                 'Z01','Z16','Z08','Z09','Z05','Z12','Z04','Z13','Z06','Z11','Z03','Z14','Z07','Z10','Z02','Z15']

In [727]:
def prediction(team1, team2):
    x = 0
    if x == 0:
        return team1, team2, 0
    else:
        return team2, team1, 1

def array_filler(num_teams):
    if   num_teams == 64:
        roundof32.append(winner)
    elif num_teams == 32:
        sweet16.append(winner)
    elif num_teams == 16:
        elite8.append(winner)
    elif num_teams == 8:
        final4.append(winner)
    elif num_teams == 4:
        finals.append(winner)
    elif num_teams == 2:
        ncaa_winner.append(winner)
        
season = 2014
teams_left = len(tourney_order)
next_round = tourney_order[:]

for round_num in range(0,6):        
        num_teams = 2**(6-round_num)
        print("----------- Number of Teams left: %s" % (num_teams))
        for i in range(0, num_teams, 2):
            
            team1 = next_round[i]
            team2 = next_round[i+1]
                        
            winner, loser, which = prediction(team1, team2)
            next_round.append(next_round[i+which])
            
            print('%s beat %s to advance to the round of %s' % 
                  (seeds['TeamName'][(seeds['Seed'] == winner)&(seeds['Season']==season)].values[0], 
                   seeds['TeamName'][(seeds['Seed'] == loser)&(seeds['Season']==season)].values[0], 
                   int(num_teams/2)))
            
            array_filler(num_teams)
                        
        # splits the list so that the only elements left are teams still alive
        # after a Region the loop next_round will be empty
        
        del next_round[0:num_teams]   

----------- Number of Teams left: 64
Virginia beat Coastal Car to advance to the round of 32
Memphis beat G Washington to advance to the round of 32
Cincinnati beat Harvard to advance to the round of 32
Michigan St beat Delaware to advance to the round of 32
North Carolina beat Providence to advance to the round of 32
Iowa St beat NC Central to advance to the round of 32
Connecticut beat St Joseph's PA to advance to the round of 32
Villanova beat WI Milwaukee to advance to the round of 32
Florida beat Albany NY to advance to the round of 32
Colorado beat Pittsburgh to advance to the round of 32
VA Commonwealth beat SF Austin to advance to the round of 32
UCLA beat Tulsa to advance to the round of 32
Ohio St beat Dayton to advance to the round of 32
Syracuse beat W Michigan to advance to the round of 32
New Mexico beat Stanford to advance to the round of 32
Kansas beat E Kentucky to advance to the round of 32
Wichita St beat Cal Poly SLO to advance to the round of 32
Kentucky beat Kansa

In [724]:
# splits the regular season by win and loss and will later combine
# this is because the df is by game not overall
reg_season_win = reg_season.groupby(['WTeamID','Season'], as_index = False).agg(sum)
reg_season_lose = reg_season.groupby(['LTeamID','Season'], as_index = False).agg(sum)


# calculates wins and losses for each team
team_wins = reg_season.groupby(['WTeamID','Season']).WTeamID.count()
team_losses = reg_season.groupby(['LTeamID','Season']).LTeamID.count()

# creat lists for wins and losses
xwin = []
for i in team_wins:
    xwin.append(i)

xloss = []
for i in team_losses:
    xloss.append(i)

'''
Reason: Undefeated
Missing 2015 season for teamId=1246 losing data 690
Missing 2014 season for teamId=1455 losing data 1705

Reason: Defeated
Missing 2015 season for teamId=1212 winning data
Missing 2015 season for teamId=1363 winning data 
'''

# inserts a 0 where the value was missing
xloss.insert(691,0)
xloss.insert(1705,0)

xwin.insert(526,0)
xwin.insert(1261,0)

In [725]:
# renames columns to make it easier to combine, merges based on team
# winning team becomes team and losing team become opponent stats
# same but flipped below for the lose regular season
# drops unnecessary columns
reg_season_win=reg_season_win.rename(columns={'WTeamID':'TeamID', 'WFGM':'FGM', 'WFGA':'FGA', 'WFGM3':'FGM3', 'WFGA3':'FGA3', 
                                          'WFTM':'FTM', 'WFTA':'FTA', 'WOR':'OR', 'WDR':'DR', 'WAst':'Ast', 'WTO':'TO', 
                                          'WStl':'Stl', 'WBlk':'Blk', 'WPF':'PF', 'LFGM':'OFGM', 'LFGA':'OFGA', 
                                          'LFGM3':'OFGM3', 'LFGA3':'OFGA3', 'LFTM':'OFTM', 'LFTA':'OFTA', 'LOR':'OOR',
                                          'LDR':'ODR', 'LAst':'OAst', 'LTO':'OTO', 'LStl':'OStl', 'LBlk':'OBlk', 'LPF':'OPF'
                                           })
reg_season_win = reg_season_win.drop(['DayNum','WScore','LTeamID','LScore','NumOT'], axis = 1)

reg_season_lose=reg_season_lose.rename(columns={'LTeamID':'TeamID', 'LFGM':'FGM', 'LFGA':'FGA', 'LFGM3':'FGM3', 'LFGA3':'FGA3', 
                                          'LFTM':'FTM', 'LFTA':'FTA', 'LOR':'OR', 'LDR':'DR', 'LAst':'Ast', 'LTO':'TO', 
                                          'LStl':'Stl', 'LBlk':'Blk', 'LPF':'PF', 'WFGM':'OFGM', 'WFGA':'OFGA', 
                                          'WFGM3':'OFGM3', 'WFGA3':'OFGA3', 'WFTM':'OFTM', 'WFTA':'OFTA', 'WOR':'OOR',
                                          'WDR':'ODR', 'WAst':'OAst', 'WTO':'OTO', 'WStl':'OStl', 'WBlk':'OBlk', 'WPF':'OPF'
                                           })
reg_season_lose = reg_season_lose.drop(['DayNum','WScore','WTeamID','LScore','NumOT'], axis = 1)

regular_season = pd.concat([reg_season_win,reg_season_lose])

# reindexs to make it look cleaner
regular_season = regular_season.reindex_axis(reg_season_win.columns, axis=1)

# Once again takes the sum of every column grouped by each team
regular_season = regular_season.groupby(['TeamID','Season'],as_index = False).agg(sum)

# creates names array for each team ID 
names = []
for team in regular_season.TeamID:
    names.append(team_names['TeamName'][team_names['TeamID']==team].values[0])

# inserts Team name, wins, losses columns
regular_season.insert(loc=0, column ='TeamName',value=names)
regular_season.insert(loc=3, column = 'W', value = xwin)
regular_season.insert(loc=4, column = 'L', value = xloss)


# creates new columns Games, field goal %, 3-point %, free throw %
regular_season.insert(loc=3, column = 'G', value = regular_season.W+regular_season.L)
regular_season.insert(loc=8, column = 'FG%', value = regular_season.FGM*100/regular_season.FGA)
regular_season.insert(loc=11, column = 'FG3%', value = regular_season.FGM3*100/regular_season.FGA3)
regular_season.insert(loc=14, column = 'FT%', value = regular_season.FTM*100/regular_season.FTA)

# views new tidy, clean data frame
regular_season

this is length3: 1755


,TeamName,TeamID,Season,G,W,L,FGM,FGA,FG%,FGM3,...,OFGA3,OFTM,OFTA,OOR,ODR,OAst,OTO,OStl,OBlk,OPF
0,Abilene Chr,1101,2014,21,2,19,427,1053,40.550807,140,...,340,385,542,218,506,327,255,147,105,392
1,Abilene Chr,1101,2015,28,7,21,600,1482,40.485830,203,...,468,437,636,281,725,362,377,164,119,463
2,Abilene Chr,1101,2016,27,9,18,643,1456,44.162088,185,...,409,478,674,232,681,347,363,138,89,531
3,Abilene Chr,1101,2017,25,9,16,611,1333,45.836459,177,...,490,397,595,266,626,340,370,151,79,431
4,Abilene Chr,1101,2018,27,12,15,689,1525,45.180328,175,...,533,449,633,268,676,325,418,176,79,477
5,Air Force,1102,2014,28,10,18,618,1451,42.591316,200,...,596,461,651,277,681,382,327,195,127,520
6,Air Force,1102,2015,29,12,17,685,1472,46.535326,208,...,645,358,519,291,611,419,359,168,105,495
7,Air Force,1102,2016,30,12,18,697,1651,42.216838,189,...,794,449,632,299,819,439,407,195,130,618
8,Air Force,1102,2017,31,10,21,751,1755,42.792023,246,...,828,413,577,255,813,491,378,195,108,638
9,Air Force,1102,2018,29,10,19,660,1627,40.565458,230,...,677,376,526,266,755,423,405,162,104,543
